In [ ]:
import pandas as pd
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I

In [ ]:
# This works
butler = Butler('LSSTCam', collections=["LSSTCam/raw/all", "LSSTCam/calib", "LSSTCam/runs/quickLook"])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()

In [ ]:
# Check of basic data
dayObs = 20250912
table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()
table.loc[207]

In [ ]:
# Check of guider data
dayObs = 20250912
guiderTable = pd.read_json(f'/project/rubintv/LSSTCam/guiders/sidecar_metadata/dayObs_{dayObs}.json').T
guiderTable = guiderTable.sort_index()
print(guiderTable.loc[207]['Alt drift (arcsec total)'], 
      guiderTable.loc[207]['Az drift (arcsec total)'])
print(float(guiderTable.loc[207]['Alt drift (arcsec total)'])*30.0, 
      float(guiderTable.loc[207]['Az drift (arcsec total)'])*30.0)
# This agrees with what's in the centroid plot.

In [ ]:
guiderTable.columns

## Get the guider drift data from RubinTV

In [ ]:
startDay = 20251026
endDay = 20251027
azs = []
els = []
rots = []
az_drifts = []
el_drifts = []
rot_drifts = []
expIds = []
dayObs = startDay
while dayObs <= endDay:
        try:
            guiderTable = pd.read_json(f'/project/rubintv/LSSTCam/guiders/sidecar_metadata/dayObs_{dayObs}.json').T
            guiderTable = guiderTable.sort_index()
            for i in range(1, len(guiderTable)+1):
                if i<100:
                    continue
                expId = int(dayObs * 1.0E5 + i)
                try:
                    expTime = float(guiderTable.loc[i]['Exposure time'])
                    az = float(guiderTable.loc[i]['Azimuth'])
                    el = float(guiderTable.loc[i]['Elevation'])
                    rot = float(guiderTable.loc[i]['Sky angle'])
                    az_drift = float(guiderTable.loc[i]['Az drift (arcsec total)'])
                    el_drift = float(guiderTable.loc[i]['Alt drift (arcsec total)'])
                    rot_drift = float(guiderTable.loc[i]['Rotator drift (arcsec total)'])
                    az_drift *= expTime
                    el_drift *= expTime
                    rot_drift *= expTime
                    data = np.array([az, el, rot, az_drift, el_drift, rot_drift])
                    if np.isnan(data).any():
                        continue
                    expIds.append(expId)
                    azs.append(az)
                    els.append(el)
                    rots.append(rot)
                    az_drifts.append(az_drift)
                    el_drifts.append(el_drift)
                    rot_drifts.append(rot_drift)
                except:
                    continue
            dayObs = calcNextDay(dayObs) 
        except:
            print(f"{dayObs} failed")
            dayObs = calcNextDay(dayObs)
            continue
        print(f" Done with {dayObs}")
print(len(expIds))

In [ ]:
# Check the data against the RubinTV plots
thisExpId = 2025091200207
for i, expId in enumerate(expIds):
    if expId == thisExpId:
        print(expId, az_drifts[i], el_drifts[i])

## Plot the guider errors vs AltAz.

In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle(f"Guider drifts {startDay} - {endDay}")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(els, el_drifts, c=azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('El drift arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(azs, el_drifts ,c=els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('El drift arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(els, az_drifts, c=azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Az drift arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(azs, az_drifts,c=els, cmap=plt.cm.coolwarm)
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Az drift arcsec')
plt.savefig(f"/home/cslage/DATA/Guider_Drifts_{startDay}_{endDay}.png")


In [ ]:
def DeltaAltAz(ra, dec, pressure, hum, temperature, wl, time1, time2):
    # This calculates the change in AltAz during an exposure
    # given the RA/Dec and other variables
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    return [azChange, elChange]

In [ ]:
# Below is the wavelength center point for the LSST filters:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}
detector = camera['R22_S11']
bbox = detector.getBBox()

def CalculateDrift(expId):
    rawExp = butler.get('raw', detector=94, exposure=expId, instrument=instrument)
    md = rawExp.getMetadata()
    filter = md['FILTBAND']
    wl = wavelengths[filter] * u.angstrom
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY'] / 100.0
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai')
    raPoint = md['RA']
    decPoint = md['DEC']
    el = md['ELSTART']
    #print(f"We think telescope is pointed at (RA, Dec), ({raPoint:.6f}, {decPoint:.6f})")
    [azChangePoint, elChangePoint] = DeltaAltAz(raPoint, decPoint, pressure, hum, temperature, wl, time1, time2)
    calExp = butler.get('preliminary_visit_image', detector=94, visit=expId, instrument=instrument)
    cWcs = calExp.getWcs()
    if not cWcs:
        return None
    calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    raReal = calExpSkyCenter.getRa().asDegrees()
    decReal = calExpSkyCenter.getDec().asDegrees()
    deltaRa = (raReal - raPoint) * 3600.0
    deltaDec = (decReal - decPoint) * 3600.0
    #print(f"Telescope is actually pointed at (RA, Dec), ({raReal:.6f}, {decReal:.6f})")
    #print(f" Pointing error in RA, Dec is ({deltaRa:.1f}, {deltaDec:.1f}) arcseconds")
    [azChangeReal, elChangeReal] = DeltaAltAz (raReal, decReal, pressure, hum, temperature, wl, time1, time2)
    azDrift = azChangeReal - azChangePoint
    elDrift = elChangeReal - elChangePoint
    totalDrift = np.sqrt(elDrift**2 + (azDrift * np.cos(el * np.pi / 180.0))**2)
    #print(f" For {expId}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds, Total drift = {totalDrift:.2f} arcseconds.")
    return [azDrift, elDrift]

## Get the expected pointing model error drifts

In [ ]:
ptg_az_drifts = []
ptg_el_drifts = []
ptg_expIds = []
for expId in expIds:
    try:
        [azDrift, elDrift] = CalculateDrift(expId)
        ptg_az_drifts.append(azDrift)
        ptg_el_drifts.append(elDrift)
        ptg_expIds.append(expId)
        print(f"{expId} succeeded!")
    except:
        print(f"{expId} failed!")
        continue
filename = f"/home/cslage/DATA/guider_drifts_2_{startDay}_{endDay}.pkl"
with open(filename, 'wb') as f:
    pkl.dump([ptg_expIds, ptg_az_drifts, ptg_el_drifts], f)
    

In [ ]:
startDay = 20250816
endDay = 20250915

filename = f"/home/cslage/DATA/guider_drifts_2_{startDay}_{endDay}.pkl"
print(filename)
with open(filename, 'rb') as f:
    [ptg_expIds, ptg_az_drifts, ptg_el_drifts] = pkl.load(f)
print(len(ptg_expIds), len(ptg_az_drifts), len(ptg_el_drifts))

## Combine the two sets of lists into a common set where both have data.  Also cut on pointing model drifts which are anomalously large. I think these are caused by LUT experiments

In [ ]:
ptg_az_drifts_2 = []
ptg_el_drifts_2 = []
ptg_expIds_2 = []

guider_az_drifts = []
guider_el_drifts = []
guider_total_drifts = []
ptg_azs = []
ptg_els = []
ptg_total_drifts = []
for i, expId in enumerate(expIds):
    for j, ptg_expId in enumerate(ptg_expIds):
        if (ptg_expId == expId):
            if (abs(ptg_az_drifts[j]) < 0.8) \
            and (abs(ptg_el_drifts[j]) < 0.8):
                ptg_azs.append(azs[i])
                ptg_els.append(els[i])
                guider_az_drifts.append(az_drifts[i])
                guider_el_drifts.append(el_drifts[i])
                guider_total_drift = np.sqrt(el_drifts[i]**2 +
                             (az_drifts[i] * np.cos(els[i] * np.pi / 180.0))**2)
                guider_total_drifts.append(guider_total_drift)
                ptg_az_drifts_2.append(ptg_az_drifts[j])
                ptg_el_drifts_2.append(ptg_el_drifts[j])
                ptg_expIds_2.append(ptg_expIds[j])
                ptg_total_drift = np.sqrt(ptg_el_drifts[j]**2 +
                             (ptg_az_drifts[j] * np.cos(els[i] * np.pi / 180.0))**2)
                ptg_total_drifts.append(ptg_total_drift)
            else:
                continue
                #print(f"{ptg_expId} drifts > 0.8")
        else:
            continue

print(len(ptg_expIds_2), len(ptg_az_drifts_2), len(ptg_el_drifts_2), 
      len(ptg_azs), len(ptg_els), len(guider_az_drifts), len(guider_el_drifts),
     len(guider_total_drifts), len(ptg_total_drifts))    

## Add the pointing error drifts to the AltAz plots

In [ ]:
plt.figure(figsize=(8,8))
plt.suptitle(f"Guider drifts {startDay} - {endDay}")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(els, el_drifts, c=azs, cmap=plt.cm.coolwarm)
plt.scatter(ptg_els, ptg_el_drifts_2, marker='x', color='black')
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('El drift arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(azs, el_drifts ,c=els, cmap=plt.cm.coolwarm)
plt.scatter(ptg_azs, ptg_el_drifts_2, marker='x', color='black')
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('El drift arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(els, az_drifts, c=azs, cmap=plt.cm.coolwarm)
plt.scatter(ptg_els, ptg_az_drifts_2, marker='x', color='black')
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Az drift arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(azs, az_drifts,c=els, cmap=plt.cm.coolwarm)
plt.scatter(ptg_azs, ptg_az_drifts_2, marker='x', color='black')
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Az drift arcsec')
plt.savefig(f"/home/cslage/DATA/Guider_Drifts_Pointing_{startDay}_{endDay}.png")

## Plot a time series for the multiple nights

In [ ]:
plt.figure(figsize=(12,5))
plt.suptitle(f"Guider drifts vs Pointing Error drifts {startDay}-{endDay}")
plt.subplot(1,2,1)
plt.title("Azimuth drift/exposure")
plt.plot(ptg_az_drifts_2, marker='x', ms=0.1, label="Ptg error")
plt.plot(guider_az_drifts, marker='o', ms=0.1, alpha=0.3, label="Guider")
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.legend(loc = 'lower left')
plt.subplot(1,2,2)
plt.title("Elevation drift/exposure")
plt.plot(ptg_el_drifts_2, marker='x', ms=0.1)
plt.plot(guider_el_drifts, marker='o', ms=0.1, alpha=0.3)
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.savefig(f"/home/cslage/DATA/Guider_Drifts_Pointing_2_{startDay}_{endDay}.png")

In [ ]:
plt.figure(figsize=(10,5))
plt.suptitle(f"Guider drifts vs Pointing Error drifts {startDay}-{endDay}")
plt.title("Total drift/exposure")
plt.plot(ptg_total_drifts, marker='x', ms=0.1, label="Ptg error")
plt.plot(guider_total_drifts, marker='o', ms=0.1, alpha=0.3, label="Guider")
plt.xlabel("Exposure list index")
plt.ylabel("Drift/exposure (arcsec)")
plt.legend(loc = 'lower left')
plt.savefig(f"/home/cslage/DATA/Guider_Drifts_Total_{startDay}_{endDay}.png")

## Look at the drift ratios.

In [ ]:
az_ratio_limits = (-2.0, 4.0)
el_ratio_limits = (-10.0, 10.0)
az_ratios = np.array(guider_az_drifts) / np.array(ptg_az_drifts_2)
az_ratios = [ratio for ratio in az_ratios \
             if (ratio > az_ratio_limits[0]) \
             and (ratio < az_ratio_limits[1])]
el_ratios = np.array(guider_el_drifts) / np.array(ptg_el_drifts_2)
el_ratios = [ratio for ratio in el_ratios \
             if (ratio > el_ratio_limits[0]) \
             and (ratio < el_ratio_limits[1])]
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.title("Azimuth ratio Guider drift / Pointing error drift")
plt.hist(az_ratios, bins=50, range=az_ratio_limits)
plt.text(-2, 80, f"Median={np.median(az_ratios):.2f}")
plt.text(-2, 75, f"StDev={np.std(az_ratios):.2f}")
plt.subplot(1,2,2)
plt.title("Elevation ratio Guider drift / Pointing error drift")
plt.hist(el_ratios, bins=50, range=el_ratio_limits)
plt.text(-8, 40, f"Median={np.median(el_ratios):.2f}")
plt.text(-8, 35, f"StDev={np.std(el_ratios):.2f}")
plt.savefig(f"/home/cslage/DATA/Guider_Drifts_Ratios_{startDay}_{endDay}.png")

## Another way to combine the data, keeping it separate by night.

In [ ]:
dataDict = {}
dayObss = [20251026, 20251027]#[20250825, 20250826, 20250827, 20250828, 20250902, 20250906, \
#           20250907, 20250909, 20250910, 20250911, 20250915]
for dayObs in dayObss:
    guider_az_drifts = []
    guider_el_drifts = []
    guider_total_drifts = []
    these_az_drifts = []
    these_el_drifts = []
    these_total_drifts = []
    these_expIds = []
    for i, expId in enumerate(expIds):
        for j, ptg_expId in enumerate(ptg_expIds):
            ptg_dayObs = int(ptg_expId / 1E5)
            #print(dayObs, ptg_dayObs)
            if (ptg_expId == expId) and (ptg_dayObs == dayObs):
                if (abs(ptg_az_drifts[j]) < 0.8) \
                and (abs(ptg_el_drifts[j]) < 0.8):
                    ptg_azs.append(azs[i])
                    ptg_els.append(els[i])
                    guider_az_drifts.append(az_drifts[i])
                    guider_el_drifts.append(el_drifts[i])
                    guider_total_drift = np.sqrt(el_drifts[i]**2 +
                             (az_drifts[i] * np.cos(els[i] * np.pi / 180.0))**2)
                    guider_total_drifts.append(guider_total_drift)
                    these_az_drifts.append(ptg_az_drifts[j])
                    these_el_drifts.append(ptg_el_drifts[j])
                    this_total_drift = np.sqrt(ptg_el_drifts[j]**2 +
                             (ptg_az_drifts[j] * np.cos(els[i] * np.pi / 180.0))**2)
                    these_total_drifts.append(this_total_drift)
                    these_expIds.append(ptg_expIds[j])
                else:
                    continue
            else:
                continue
    dataDict[dayObs] = [these_expIds, ptg_azs, ptg_els, \
                       these_az_drifts, these_el_drifts, \
                       guider_az_drifts, guider_el_drifts, \
                        guider_total_drifts, these_total_drifts]
    print(dayObs, len(these_expIds))



## Plot the time series night by night.

In [ ]:
for dayObs in dayObss:
    [these_expIds, ptg_azs, ptg_els, \
                       these_az_drifts, these_el_drifts, \
                       guider_az_drifts, guider_el_drifts, \
                        guider_total_drifts, these_total_drifts] = dataDict[dayObs] 
    seqNums = np.array(these_expIds) - dayObs * 1E5
    
    plt.figure(figsize=(12,5))
    plt.suptitle(f"Guider drifts vs Pointing Error drifts {dayObs}")
    plt.subplot(1,2,1)
    plt.title("Azimuth drift/exposure")
    plt.plot(seqNums, these_az_drifts, marker='x', ms=2.0, label="Ptg error")
    plt.plot(seqNums, guider_az_drifts, marker='o', ms=2.0, alpha=0.3, label="Guider")
    plt.xlabel("SeqNum")
    plt.ylabel("Drift/exposure (arcsec)")
    plt.legend(loc = 'lower left')
    plt.subplot(1,2,2)
    plt.title("Elevation drift/exposure")
    plt.plot(seqNums, these_el_drifts, marker='x', ms=2.0)
    plt.plot(seqNums, guider_el_drifts, marker='o', ms=2.0, alpha=0.3)
    plt.xlabel("SeqNum")
    plt.ylabel("Drift/exposure (arcsec)")
    plt.savefig(f"/home/cslage/DATA/Guider_Drifts_Pointing_3_{dayObs}.png")

In [ ]:
for dayObs in dayObss:
    [these_expIds, ptg_azs, ptg_els, \
                       these_az_drifts, these_el_drifts, \
                       guider_az_drifts, guider_el_drifts, \
                        guider_total_drifts, these_total_drifts] = dataDict[dayObs] 
    seqNums = np.array(these_expIds) - dayObs * 1E5
    
    plt.figure(figsize=(5,5))
    plt.suptitle(f"Guider drifts vs Pointing Error drifts {dayObs}")
    plt.title("Total drift/exposure")
    plt.plot(seqNums, np.array(these_total_drifts) + 0.15, marker='x', ms=2.0, label="Ptg error")
    plt.plot(seqNums, guider_total_drifts, marker='o', ms=2.0, alpha=0.3, label="Guider")
    plt.xlabel("SeqNum")
    plt.ylabel("Drift/exposure (arcsec)")
    plt.legend(loc = 'lower left')
    plt.savefig(f"/home/cslage/DATA/Guider_Drifts_Pointing_4_{dayObs}.png")

In [ ]:
dayObs = 20251023
table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()
expIds = []
azDrifts = []
elDrifts = []
for i in range(1, len(table)+1):
    expId = int(dayObs * 1.0E5 + i)
    try:
        [azDrift, elDrift] = CalculateDrift(expId)
        azDrifts.append(azDrift)
        elDrifts.append(elDrift)
        expIds.append(expId)
    except:
        continue
    if expId % 10 == 0:
        print(f"Finished {expId}")
    


In [ ]:
expIds = np.array(expIds)
seqNums = expIds - dayObs * 1E5
plt.scatter(seqNums, azDrifts, label='Az')
plt.scatter(seqNums, elDrifts, label='El')
plt.ylim(-1.0, 1.0)
plt.legend()